In [1]:
!pip install kaggle torch torchvision pillow tqdm

from google.colab import files
files.upload()   # Upload your kaggle.json here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"prateekshetty28","key":"253cf17c53d9ab0465b6311ee47cdce7"}'}

In [2]:
import os
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -q chest-xray-pneumonia.zip


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [00:14<00:00, 37.1MB/s]
100% 2.29G/2.29G [00:14<00:00, 168MB/s] 


In [4]:
!kaggle datasets download -d abhranta/brain-tumor-mri-dataset # replace if needed
!unzip -q brain-tumor-mri-dataset.zip


403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/datasets/metadata/abhranta/brain-tumor-mri-dataset
unzip:  cannot find or open brain-tumor-mri-dataset.zip, brain-tumor-mri-dataset.zip.zip or brain-tumor-mri-dataset.zip.ZIP.


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [6]:
def create_loader(path, img_size=224, batch=32):
    train_tf = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor()
    ])

    test_tf = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor()
    ])

    train_ds = datasets.ImageFolder(os.path.join(path, "train"), transform=train_tf)
    val_ds   = datasets.ImageFolder(os.path.join(path, "val"), transform=test_tf)
    test_ds  = datasets.ImageFolder(os.path.join(path, "test"), transform=test_tf)

    train_loader = DataLoader(train_ds, batch, shuffle=True)
    val_loader   = DataLoader(val_ds, batch, shuffle=False)
    test_loader  = DataLoader(test_ds, batch, shuffle=False)

    return train_loader, val_loader, test_loader, train_ds.classes


In [7]:
pneu_train, pneu_val, pneu_test, pneu_classes = create_loader("chest_xray")
pneu_classes


['NORMAL', 'PNEUMONIA']

In [8]:
def train_model(model, train_loader, val_loader, epochs=5, lr=1e-4):
    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total, correct = 0, 0
        loop = tqdm(train_loader)

        for imgs, labels in loop:
            imgs, labels = imgs.to(device), labels.to(device)

            opt.zero_grad()
            out = model(imgs)
            loss = loss_fn(out, labels)
            loss.backward()
            opt.step()

            preds = out.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            loop.set_description(f"Epoch {epoch+1}/{epochs}")
            loop.set_postfix(loss=loss.item(), acc=correct/total)

        # validation
        model.eval()
        v_total, v_correct = 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                out = model(imgs)
                preds = out.argmax(1)
                v_correct += (preds == labels).sum().item()
                v_total += labels.size(0)

        print(f"VAL ACC: {v_correct/v_total:.4f}")

    return model


In [9]:
pneu_model = models.densenet121(weights='IMAGENET1K_V1')
pneu_model.classifier = nn.Linear(pneu_model.classifier.in_features, len(pneu_classes))

pneu_model = train_model(pneu_model, pneu_train, pneu_val, epochs=15)
torch.save(pneu_model.state_dict(), "pneumonia_model.pt")


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:00<00:00, 132MB/s]
Epoch 1/5:   6%|▌         | 9/163 [03:00<51:32, 20.08s/it, acc=0.868, loss=0.271]


KeyboardInterrupt: 

In [ ]:
brain_train, brain_val, brain_test, brain_classes = create_loader("Brain Tumor MRI Dataset")

brain_model = models.resnet50(weights='IMAGENET1K_V1')
brain_model.fc = nn.Linear(brain_model.fc.in_features, len(brain_classes))

brain_model = train_model(brain_model, brain_train, brain_val, epochs=15)
torch.save(brain_model.state_dict(), "brain_model.pt")


In [ ]:
from google.colab import files

files.download("pneumonia_model.pt")
files.download("brain_model.pt")
